#import modules

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string


#Loading and preprocessing the data

In [32]:
df = pd.read_csv('Tamil_first_ready_for_sentiment.csv', sep='\t')
df.rename(columns = {'Negative ':'sentiment',
                              ' Enna da ellam avan seyal  Mari iruku':'comment'},
                              inplace = True)

#change the value of positive and negative and mixfelling to 0 1 2
df['sentiment'] = df.sentiment.str.replace('Negative', '0')
df['sentiment'] = df.sentiment.str.replace('Positive', '1')
df['sentiment'] = df.sentiment.str.replace('Mixed_feelings', '2')


import string

#defining the function to remove punctuation
def remove_punctuation(text):
    
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#storing the puntuation free text
df['clean_comment']= df['comment'].apply(lambda x:remove_punctuation(x))

df.head()

df['comment_lower']= df['clean_comment'].apply(lambda x: x.lower())

#defining function for tokenization
import re
def tokenization(text):
    tokens = re.split('W+',text)
    return tokens
#applying function to the column
df['comment_tokenied'] = df['comment_lower'].apply(lambda x: tokenization(x))

df['comment_tokenied'][:5]

import nltk

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

df['no_stopwords']= df['comment_tokenied'].apply(lambda x:remove_stopwords(x))

#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

#defining a function for stemming
def stemming(text):
  stem_text = [porter_stemmer.stem(word) for word in text]
  return stem_text

df['comment_stemmed'] = df['no_stopwords'].apply(lambda x: stemming(x))

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text


# Python program to convert a list to string
def listToString(s):

	# initialize an empty string
	str1 = ""

	# traverse in the string
	for ele in s:
		str1 += ele

	# return string
	return str1  

df['comment_lemmatized'] = df['no_stopwords'].apply(lambda x:lemmatizer(x))
df['comment_lemmatized_new']= df['comment_lemmatized'].apply(lambda x: listToString(x))



X = df['comment_lemmatized_new']
y = df['sentiment']
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,sentiment,comment,clean_comment,comment_lower,comment_tokenied,no_stopwords,comment_stemmed,comment_lemmatized,comment_lemmatized_new
0,0,This movei is just like ellam avan seyal,This movei is just like ellam avan seyal,this movei is just like ellam avan seyal,[ this movei is just like ellam avan seyal],[ this movei is just like ellam avan seyal],[ this movei is just like ellam avan sey],[ this movei is just like ellam avan seyal],this movei is just like ellam avan seyal
1,1,Padam vanthathum 13k dislike pottavaga yellam...,Padam vanthathum 13k dislike pottavaga yellam...,padam vanthathum 13k dislike pottavaga yellam...,[ padam vanthathum 13k dislike pottavaga yella...,[ padam vanthathum 13k dislike pottavaga yella...,[ padam vanthathum 13k dislike pottavaga yella...,[ padam vanthathum 13k dislike pottavaga yella...,padam vanthathum 13k dislike pottavaga yellam...
2,1,Neraya neraya neraya... ... V era level...thala,Neraya neraya neraya V era levelthala,neraya neraya neraya v era levelthala,[ neraya neraya neraya v era levelthala],[ neraya neraya neraya v era levelthala],[ neraya neraya neraya v era levelthala],[ neraya neraya neraya v era levelthala],neraya neraya neraya v era levelthala
3,1,wow thavala sema mass....padam oru pundaikum ...,wow thavala sema masspadam oru pundaikum aagathu,wow thavala sema masspadam oru pundaikum aagathu,[ wow thavala sema masspadam oru pundaikum aag...,[ wow thavala sema masspadam oru pundaikum aag...,[ wow thavala sema masspadam oru pundaikum aag...,[ wow thavala sema masspadam oru pundaikum aag...,wow thavala sema masspadam oru pundaikum aagathu
4,0,Andha 19 k unlike panavangaluku kolandha pora...,Andha 19 k unlike panavangaluku kolandha pora...,andha 19 k unlike panavangaluku kolandha pora...,[ andha 19 k unlike panavangaluku kolandha por...,[ andha 19 k unlike panavangaluku kolandha por...,[ andha 19 k unlike panavangaluku kolandha por...,[ andha 19 k unlike panavangaluku kolandha por...,andha 19 k unlike panavangaluku kolandha pora...


# using the train test split function

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

# Extracting features from the training data using a sparse vectorizer


In [55]:
vectorizer = TfidfVectorizer(
    sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english")

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print(y_train[156])



1 


#Creating Helper Functions


In [35]:
# Helper function for merging new history objects with older ones
def append_history(losses, val_losses, accuracy, val_accuracy, history):
    losses = losses + history.history["loss"]
    val_losses = val_losses + history.history["val_loss"]
    accuracy = accuracy + history.history["binary_accuracy"]
    val_accuracy = val_accuracy + history.history["val_binary_accuracy"]
    return losses, val_losses, accuracy, val_accuracy


# Plotter function
def plot_history(losses, val_losses, accuracies, val_accuracies):
    plt.plot(losses)
    plt.plot(val_losses)
    plt.legend(["train_loss", "val_loss"])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.plot(accuracies)
    plt.plot(val_accuracies)
    plt.legend(["train_accuracy", "val_accuracy"])
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()

#Creating the Model


In [65]:
def create_model():
    model = keras.models.Sequential(
        [
            layers.Input(shape=(150,)),
            layers.Embedding(input_dim=3000, output_dim=128),
            layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
            layers.GlobalMaxPool1D(),
            layers.Dense(20, activation="relu"),
            layers.Dropout(0.5),
            layers.Dense(1, activation="sigmoid"),
        ]
    )
    model.summary()
    return model



#Training on the entire dataset


In [66]:
def train_full_model():
    model = create_model()
    model.compile(
        loss="binary_crossentropy",
        optimizer="rmsprop",
        metrics=[
            keras.metrics.BinaryAccuracy(),
            keras.metrics.FalseNegatives(),
            keras.metrics.FalsePositives(),
        ],
    )

    # We will save the best model at every epoch and load the best one for evaluation on the test set
    history = model.fit(
        X_train,
        y_train,
        epochs=20,
        validation_data=(X_test, y_test),
        callbacks=[
            keras.callbacks.EarlyStopping(patience=4, verbose=1),
            keras.callbacks.ModelCheckpoint(
                "FullModelCheckpoint.h5", verbose=1, save_best_only=True
            ),
        ],
    )

    # Plot history
    plot_history(
        history.history["loss"],
        history.history["val_loss"],
        history.history["binary_accuracy"],
        history.history["val_binary_accuracy"],
    )

    # Loading the best checkpoint
    model = keras.models.load_model("FullModelCheckpoint.h5")

    print("-" * 100)
    print(
        "Test set evaluation: ",
        model.evaluate(X_test, y_test, verbose=0, return_dict=True),
    )
    print("-" * 100)
    return model

train_full_model()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 150, 128)          384000    
                                                                 
 bidirectional_7 (Bidirectio  (None, 150, 64)          41216     
 nal)                                                            
                                                                 
 global_max_pooling1d_7 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_14 (Dense)            (None, 20)                1300      
                                                                 
 dropout_7 (Dropout)         (None, 20)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                

Epoch 1/20


ValueError: ignored